# **Image Classification using SVM with TensorFlow**
This notebook demonstrates how to use **Support Vector Machines (SVM)** for image classification by extracting features from a pre-trained **MobileNetV2** model in TensorFlow.

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt


## **1. Load and Preprocess Data**
- We use **Fashion MNIST**, a dataset of grayscale 28x28 images.
- Images are **normalized** to [0,1] and resized to **96x96** to match MobileNetV2 input size.
- Convert grayscale images to **3 channels** (RGB).

In [ ]:
# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Resize images to match MobileNetV2 input size
x_train_resized = np.stack([tf.image.resize(img[..., np.newaxis], (96, 96)).numpy() for img in x_train])
x_test_resized = np.stack([tf.image.resize(img[..., np.newaxis], (96, 96)).numpy() for img in x_test])

# Convert grayscale images to 3 channels
x_train_resized = np.repeat(x_train_resized, 3, axis=-1)
x_test_resized = np.repeat(x_test_resized, 3, axis=-1)

## **2. Feature Extraction using MobileNetV2**
We load a **pre-trained MobileNetV2 model** (without its top classification layer) to extract meaningful features from images.

In [ ]:
# Load pre-trained MobileNetV2 model without the classification head
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(96, 96, 3))

# Extract features using Global Average Pooling
x = base_model.output
x = GlobalAveragePooling2D()(x)
feature_extractor = Model(inputs=base_model.input, outputs=x)

# Extract features for train and test sets
train_features = feature_extractor.predict(x_train_resized)
test_features = feature_extractor.predict(x_test_resized)

## **3. Train SVM Classifier**
Now, we train an **SVM classifier** using the extracted features.

In [ ]:
# Train SVM classifier
svm_clf = SVC(kernel='linear', C=1.0)
svm_clf.fit(train_features, y_train)

## **4. Evaluate Model Performance**
We assess the model's accuracy and performance using a confusion matrix and classification report.

In [ ]:
# Make predictions
y_pred = svm_clf.predict(test_features)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

## **5. Visualize Some Predictions**
We display a few test images with their predicted and actual labels.

In [ ]:
# Visualize some test images with predicted labels
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for ax, image, label, pred in zip(axes.ravel(), x_test[:10], y_test[:10], y_pred[:10]):
    ax.set_axis_off()
    ax.imshow(image, cmap='gray')
    ax.set_title(f"Label: {label}\nPred: {pred}")

plt.show()

## **Conclusion**
We successfully used **MobileNetV2** to extract deep features and trained an **SVM classifier** for image classification.